# 0. Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

In [9]:
!pip freeze opencv-python

absl-py==0.12.0
aiobotocore==2.2.0
aiohttp==3.8.1
aioitertools==0.8.0
aiosignal==1.2.0
alembic==1.4.1
anyio==3.4.0
appnope==0.1.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asgiref==3.4.1
astroid==2.7.3
astunparse==1.6.3
async-generator==1.10
async-timeout==4.0.2
attrs==21.4.0
autopep8==1.6.0
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.10.0
black==21.12b0
bleach==4.1.0
botocore==1.24.21
branca==0.4.2
cachetools==4.2.4
category-encoders==2.3.0
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.10
chromedriver-binary==77.0.3865.40.0
click==8.0.3
cloudpickle==2.0.0
colorama==0.4.4
commonmark==0.9.1
coverage==6.2
cryptography==36.0.1
cv==1.0.0
cycler==0.11.0
Cython==0.29.26
dask==2021.12.0
databricks-cli==0.16.2
deap==1.3.1
debugpy==1.5.1
decorator==5.1.0
defusedxml==0.7.1
dill==0.3.4
docker==5.0.3
docutils==0.18.1
entrypoints==0.3
fastapi==0.70.1
flake8==4.0.1
Flask==2.0.2
Flask-Cors==3.0.10
flatbuffers==2.0
folium==0.12.1.post1
fonttools==4.28.5
frozenlist==1.2.0
fsspec==20

In [4]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [ ]:
import csv
import os
import numpy as np

In [5]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [8]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        success, img = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        
        cv2.imshow('Video/image', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()        

In [12]:
body_LandM = results.pose_landmarks.landmark
body_LandM

[x: 0.5388952493667603
y: 0.8425969481468201
z: -1.2606157064437866
visibility: 0.9969443678855896
, x: 0.5669658184051514
y: 0.7567297220230103
z: -1.2362534999847412
visibility: 0.9943651556968689
, x: 0.5837820768356323
y: 0.7557867169380188
z: -1.2364282608032227
visibility: 0.9962465763092041
, x: 0.5999181270599365
y: 0.7547294497489929
z: -1.2367522716522217
visibility: 0.9945040345191956
, x: 0.5134066343307495
y: 0.747094452381134
z: -1.2396361827850342
visibility: 0.9952086210250854
, x: 0.4938947260379791
y: 0.7395344376564026
z: -1.2391276359558105
visibility: 0.9964960217475891
, x: 0.4752212166786194
y: 0.7324678897857666
z: -1.2393311262130737
visibility: 0.9953259825706482
, x: 0.6254333853721619
y: 0.7461873292922974
z: -0.8416920900344849
visibility: 0.9967359900474548
, x: 0.44766825437545776
y: 0.7176716327667236
z: -0.8320621252059937
visibility: 0.9972515106201172
, x: 0.5691941976547241
y: 0.8994100689888
z: -1.0868546962738037
visibility: 0.9956660866737366
, x:

In [13]:
len(body_LandM)

33

In [30]:
num_coords = len(body_LandM)
num_coords

33

In [32]:
lndmrks = ['class']
for val in range(1, num_coords+1):
    lndmrks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']

In [36]:
lndmrks[-4]

'x33'

In [37]:
with open('joints-coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(lndmrks)

In [55]:
class_name = "bench press"

In [59]:
cap = cv2.VideoCapture('bench_old.mp4')
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while True:
        success, img = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
    
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # 1. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # 2. Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
           
            # get  rows
            row = pose_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('joints-coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Feed', img)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# . Train Custom Model Using Scikit Learn

# Read in Collected Data and Process

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [62]:
df = pd.read_csv('joints-coords.csv')

In [63]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,squat,0.358162,0.174292,0.015589,0.999839,0.379550,0.159497,-0.042690,0.999748,0.384970,...,0.289977,0.445370,0.453131,1.007795,-0.326474,0.871862,0.495371,0.972474,0.120184,0.408548
1,squat,0.358150,0.174591,0.010393,0.999835,0.380625,0.159594,-0.046573,0.999742,0.386129,...,0.291553,0.452937,0.451660,1.009552,-0.325672,0.876209,0.478827,0.977527,0.129795,0.418847
2,squat,0.358151,0.174985,0.012355,0.999834,0.380877,0.159911,-0.043117,0.999741,0.386503,...,0.244928,0.463866,0.453311,1.009398,-0.325803,0.879114,0.474191,0.978115,0.089130,0.430731
3,squat,0.358587,0.175876,0.004426,0.999826,0.382015,0.160352,-0.050418,0.999730,0.387816,...,0.250726,0.477479,0.451649,1.014540,-0.312747,0.884222,0.489684,0.978150,0.093740,0.446947
4,squat,0.356288,0.179435,0.008269,0.999831,0.380355,0.162197,-0.047070,0.999740,0.386257,...,0.250075,0.485401,0.451506,1.015812,-0.306980,0.888777,0.497926,0.982453,0.092918,0.457363


In [64]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
2957,bench press,0.539299,0.744408,-0.472736,0.999415,0.530732,0.726228,-0.432060,0.999321,0.536350,...,-0.151243,0.198207,0.745059,2.933283,-0.323882,0.181182,0.611281,3.234659,-0.550317,0.247210
2958,bench press,0.538365,0.744717,-0.554400,0.999452,0.527480,0.725428,-0.516069,0.999356,0.533022,...,-0.188857,0.189423,0.727993,2.486392,-0.215172,0.172208,0.599160,2.873368,-0.489541,0.239449
2959,bench press,0.535978,0.740502,-0.622523,0.999475,0.527241,0.721065,-0.581928,0.999373,0.533377,...,-0.186159,0.171213,0.745590,2.799484,-0.189726,0.156024,0.594025,3.145279,-0.585576,0.216906
2960,bench press,0.531320,0.725892,-0.645791,0.999497,0.527084,0.712721,-0.608874,0.999389,0.533163,...,-0.208868,0.161926,0.725595,2.550339,-0.166338,0.148305,0.589198,2.957851,-0.533965,0.206059
2961,bench press,0.528945,0.709785,-0.619437,0.999498,0.519371,0.705526,-0.575168,0.999393,0.523837,...,-0.138099,0.182924,0.670164,1.749266,0.256550,0.144882,0.602379,2.142361,-0.281996,0.224807


In [67]:
df[df['class']=='squat']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,squat,0.358162,0.174292,0.015589,0.999839,0.379550,0.159497,-0.042690,0.999748,0.384970,...,0.289977,0.445370,0.453131,1.007795,-0.326474,0.871862,0.495371,0.972474,0.120184,0.408548
1,squat,0.358150,0.174591,0.010393,0.999835,0.380625,0.159594,-0.046573,0.999742,0.386129,...,0.291553,0.452937,0.451660,1.009552,-0.325672,0.876209,0.478827,0.977527,0.129795,0.418847
2,squat,0.358151,0.174985,0.012355,0.999834,0.380877,0.159911,-0.043117,0.999741,0.386503,...,0.244928,0.463866,0.453311,1.009398,-0.325803,0.879114,0.474191,0.978115,0.089130,0.430731
3,squat,0.358587,0.175876,0.004426,0.999826,0.382015,0.160352,-0.050418,0.999730,0.387816,...,0.250726,0.477479,0.451649,1.014540,-0.312747,0.884222,0.489684,0.978150,0.093740,0.446947
4,squat,0.356288,0.179435,0.008269,0.999831,0.380355,0.162197,-0.047070,0.999740,0.386257,...,0.250075,0.485401,0.451506,1.015812,-0.306980,0.888777,0.497926,0.982453,0.092918,0.457363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,squat,0.628853,0.372017,-0.323906,0.999149,0.616463,0.361632,-0.267816,0.998696,0.618005,...,-0.376847,0.785696,0.639322,0.809362,0.214615,0.829575,0.466922,0.874126,-0.626715,0.949422
1119,squat,0.629917,0.377365,-0.448536,0.999148,0.617398,0.366893,-0.393038,0.998675,0.618869,...,-0.359946,0.794883,0.628129,0.804873,0.203137,0.834532,0.462654,0.871390,-0.607849,0.951865
1120,squat,0.630569,0.384733,-0.474214,0.999163,0.619056,0.374244,-0.417762,0.998701,0.620623,...,-0.406715,0.806758,0.627306,0.804229,0.123463,0.843204,0.456825,0.871136,-0.666451,0.954890
1121,squat,0.630541,0.391282,-0.478103,0.999217,0.619669,0.381221,-0.420156,0.998787,0.621509,...,-0.403410,0.817724,0.625771,0.804524,0.093670,0.851496,0.450108,0.867854,-0.649538,0.957563


In [68]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [70]:
y_test

1892       deadlift
1624       deadlift
436           squat
852           squat
313           squat
           ...     
1196       deadlift
107           squat
747           squat
2406    bench press
1024          squat
Name: class, Length: 889, dtype: object

##  Train some ML Classification Models

In [75]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [81]:
pipelines = {
    #'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [82]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [83]:
fit_models

{'rc': Pipeline(memory=None,
          steps=[('standardscaler',
                  StandardScaler(copy=True, with_mean=True, with_std=True)),
                 ('ridgeclassifier',
                  RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True,
                                  fit_intercept=True, max_iter=None,
                                  normalize=False, random_state=None,
                                  solver='auto', tol=0.001))],
          verbose=False),
 'rf': Pipeline(memory=None,
          steps=[('standardscaler',
                  StandardScaler(copy=True, with_mean=True, with_std=True)),
                 ('randomforestclassifier',
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None, max_features='auto',
                                         max_leaf_nodes=None, max_samples=None,
                      

In [84]:
fit_models['rc'].predict(X_test)

array(['deadlift', 'deadlift', 'squat', 'squat', 'squat', 'deadlift',
       'squat', 'deadlift', 'squat', 'bench press', 'squat', 'squat',
       'squat', 'squat', 'squat', 'deadlift', 'deadlift', 'deadlift',
       'deadlift', 'deadlift', 'squat', 'bench press', 'deadlift',
       'squat', 'squat', 'squat', 'squat', 'deadlift', 'deadlift',
       'deadlift', 'deadlift', 'squat', 'deadlift', 'squat',
       'bench press', 'deadlift', 'bench press', 'squat', 'squat',
       'squat', 'bench press', 'deadlift', 'bench press', 'squat',
       'deadlift', 'deadlift', 'squat', 'deadlift', 'deadlift',
       'bench press', 'squat', 'bench press', 'bench press', 'squat',
       'deadlift', 'bench press', 'deadlift', 'squat', 'deadlift',
       'squat', 'bench press', 'deadlift', 'bench press', 'bench press',
       'squat', 'deadlift', 'deadlift', 'bench press', 'deadlift',
       'deadlift', 'bench press', 'squat', 'deadlift', 'squat',
       'deadlift', 'deadlift', 'squat', 'squat', 'bench 

##  Evaluate ours Model 

In [85]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [86]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

rc 1.0
rf 1.0
gb 0.9988751406074241


In [87]:
fit_models['rf'].predict(X_test)

array(['deadlift', 'deadlift', 'squat', 'squat', 'squat', 'deadlift',
       'squat', 'deadlift', 'squat', 'bench press', 'squat', 'squat',
       'squat', 'squat', 'squat', 'deadlift', 'deadlift', 'deadlift',
       'deadlift', 'deadlift', 'squat', 'bench press', 'deadlift',
       'squat', 'squat', 'squat', 'squat', 'deadlift', 'deadlift',
       'deadlift', 'deadlift', 'squat', 'deadlift', 'squat',
       'bench press', 'deadlift', 'bench press', 'squat', 'squat',
       'squat', 'bench press', 'deadlift', 'bench press', 'squat',
       'deadlift', 'deadlift', 'squat', 'deadlift', 'deadlift',
       'bench press', 'squat', 'bench press', 'bench press', 'squat',
       'deadlift', 'bench press', 'deadlift', 'squat', 'deadlift',
       'squat', 'bench press', 'deadlift', 'bench press', 'bench press',
       'squat', 'deadlift', 'deadlift', 'bench press', 'deadlift',
       'deadlift', 'bench press', 'squat', 'deadlift', 'squat',
       'deadlift', 'deadlift', 'squat', 'squat', 'bench 

In [88]:
y_test

1892       deadlift
1624       deadlift
436           squat
852           squat
313           squat
           ...     
1196       deadlift
107           squat
747           squat
2406    bench press
1024          squat
Name: class, Length: 889, dtype: object

In [91]:
with open('body_pose_dec.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Make Detections with best Model

In [92]:
with open('body_pose_dec.pkl', 'rb') as f:
    model = pickle.load(f)

In [101]:
model.predict(y)[0]

ValueError: could not convert string to float: 'squat'

In [119]:
cap = cv2.VideoCapture('bench_pres0.mp4')
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    
    while True:
        ret, frame = cap.read() 
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
    
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
  
            row = pose_row
            
            # Make Detections
            X = pd.DataFrame([row])
            body_pose_class = model.predict(X)[0]
            body_pose_prob = model.predict_proba(X)[0]
            print(body_pose_class, body_pose_prob)
            
            # Grab ear coords to write our detected pose 
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_pose_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_pose_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_pose_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_pose_prob[np.argmax(body_pose_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [1. 0. 0.]
bench press [0.99 0.01 0.  ]
bench press [0.99 0.01 0.  ]
bench press [0.99 0.01 0.  ]
bench press [0.9

In [107]:
# tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
# results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

AttributeError: 'NoneType' object has no attribute 'landmark'